# Lesson I5: Generics Fundamentals

**Duration**: 105-120 minutes  
**Stage**: Intermediate (Building Skills)

---

## 🎯 Learning Objectives

By the end of this lesson, you will be able to:
1. Create and use generic functions to work with multiple types
2. Define generic structs and enums with type parameters
3. Understand monomorphization and its performance implications
4. Apply generic constraints and bounds effectively
5. Build reusable, type-safe abstractions using generics

---

## 📋 Prerequisite Review

**Quick Check**: From our previous lessons:

1. How do you handle errors using Result<T,E>?
2. What's the difference between HashMap and BTreeMap?
3. How do you pattern match on Option<T>?
4. What are the benefits of using custom error types?

**Answers**: 1) Use match or ? operator, 2) HashMap is unordered/faster, BTreeMap is ordered, 3) `match opt { Some(val) => ..., None => ... }`, 4) Better error context and type safety

---

## 🧠 Key Concepts

### What are Generics?

**Generics** allow you to write code that works with multiple types:
- **Type Parameters**: Placeholder types (usually `T`, `U`, `V`)
- **Compile-Time**: Types are resolved at compile time
- **Zero-Cost**: No runtime overhead
- **Type Safety**: Maintains Rust's type safety guarantees

### Monomorphization

Rust's approach to generics:
- **Code Generation**: Compiler creates specific versions for each type used
- **No Runtime Cost**: Same performance as hand-written specific functions
- **Binary Size**: May increase due to code duplication
- **Optimization**: Each version can be optimized for its specific type

### Generic Constraints

- **Trait Bounds**: Specify what operations a type must support
- **Where Clauses**: Alternative syntax for complex bounds
- **Lifetime Parameters**: Generic over lifetimes as well as types

---

## 🔬 Live Code Exploration

### Generic Functions

In [ ]:
// Generic function fundamentals
fn generic_functions_demo() {
    println!("=== Generic Functions ===");
    
    // Simple generic function
    fn largest<T: PartialOrd + Copy>(list: &[T]) -> T {
        let mut largest = list[0];
        
        for &item in list {
            if item > largest {
                largest = item;
            }
        }
        
        largest
    }
    
    // Works with different types
    let numbers = vec![34, 50, 25, 100, 65];
    let result = largest(&numbers);
    println!("Largest number: {}", result);
    
    let chars = vec!['y', 'm', 'a', 'q'];
    let result = largest(&chars);
    println!("Largest char: {}", result);
    
    // Generic function with multiple type parameters
    fn make_pair<T, U>(first: T, second: U) -> (T, U) {
        (first, second)
    }
    
    let pair1 = make_pair(42, "hello");
    let pair2 = make_pair(3.14, true);
    
    println!("\nPairs created:");
    println!("Pair 1: {:?}", pair1);
    println!("Pair 2: {:?}", pair2);
    
    // Generic function with constraints
    fn print_and_return<T: std::fmt::Display + Clone>(value: T) -> T {
        println!("Value: {}", value);
        value.clone()
    }
    
    let number = print_and_return(42);
    let text = print_and_return("Rust".to_string());
    
    println!("\nReturned values: {} and {}", number, text);
    
    // Generic function with where clause
    fn compare_and_display<T, U>(t: &T, u: &U) -> bool
    where
        T: std::fmt::Display + PartialEq<U>,
        U: std::fmt::Display,
    {
        println!("Comparing {} and {}", t, u);
        t == u
    }
    
    let result = compare_and_display(&5, &5);
    println!("Are they equal? {}", result);
}

generic_functions_demo();

### Generic Structs

In [ ]:
// Generic structs and implementations

#[derive(Debug)]
struct Point<T> {
    x: T,
    y: T,
}

impl<T> Point<T> {
    fn new(x: T, y: T) -> Self {
        Point { x, y }
    }
    
    fn x(&self) -> &T {
        &self.x
    }
    
    fn y(&self) -> &T {
        &self.y
    }
}

// Implementation for specific types
impl Point<f32> {
    fn distance_from_origin(&self) -> f32 {
        (self.x.powi(2) + self.y.powi(2)).sqrt()
    }
}

// Generic struct with multiple type parameters
#[derive(Debug)]
struct Pair<T, U> {
    first: T,
    second: U,
}

impl<T, U> Pair<T, U> {
    fn new(first: T, second: U) -> Self {
        Pair { first, second }
    }
    
    fn first(&self) -> &T {
        &self.first
    }
    
    fn second(&self) -> &U {
        &self.second
    }
    
    // Method that transforms the pair
    fn map<V, W, F, G>(self, f: F, g: G) -> Pair<V, W>
    where
        F: FnOnce(T) -> V,
        G: FnOnce(U) -> W,
    {
        Pair {
            first: f(self.first),
            second: g(self.second),
        }
    }
}

// Generic struct with constraints
#[derive(Debug)]
struct Container<T: Clone + std::fmt::Display> {
    items: Vec<T>,
}

impl<T: Clone + std::fmt::Display> Container<T> {
    fn new() -> Self {
        Container { items: Vec::new() }
    }
    
    fn add(&mut self, item: T) {
        self.items.push(item);
    }
    
    fn display_all(&self) {
        for (i, item) in self.items.iter().enumerate() {
            println!("  Item {}: {}", i, item);
        }
    }
    
    fn duplicate_last(&mut self) -> Option<()> {
        if let Some(last) = self.items.last() {
            self.items.push(last.clone());
            Some(())
        } else {
            None
        }
    }
}

fn generic_structs_demo() {
    println!("\n=== Generic Structs ===");
    
    // Point with different types
    let integer_point = Point::new(5, 10);
    let float_point = Point::new(1.0, 4.0);
    
    println!("Integer point: {:?}", integer_point);
    println!("Float point: {:?}", float_point);
    println!("Distance from origin: {:.2}", float_point.distance_from_origin());
    
    // Pair with different types
    let mixed_pair = Pair::new("Hello", 42);
    println!("\nMixed pair: {:?}", mixed_pair);
    
    // Transform the pair
    let transformed = mixed_pair.map(
        |s| s.len(),
        |n| n as f64 * 2.5
    );
    println!("Transformed pair: {:?}", transformed);
    
    // Container with constraints
    let mut string_container = Container::new();
    string_container.add("Rust".to_string());
    string_container.add("is".to_string());
    string_container.add("awesome".to_string());
    
    println!("\nString container:");
    string_container.display_all();
    
    string_container.duplicate_last();
    println!("\nAfter duplicating last:");
    string_container.display_all();
    
    let mut number_container = Container::new();
    number_container.add(1);
    number_container.add(2);
    number_container.add(3);
    
    println!("\nNumber container:");
    number_container.display_all();
}

generic_structs_demo();

### Generic Enums

In [ ]:
// Generic enums and advanced patterns

#[derive(Debug)]
enum Result<T, E> {
    Ok(T),
    Err(E),
}

impl<T, E> Result<T, E> {
    fn is_ok(&self) -> bool {
        matches!(self, Result::Ok(_))
    }
    
    fn is_err(&self) -> bool {
        matches!(self, Result::Err(_))
    }
    
    fn map<U, F>(self, f: F) -> Result<U, E>
    where
        F: FnOnce(T) -> U,
    {
        match self {
            Result::Ok(value) => Result::Ok(f(value)),
            Result::Err(error) => Result::Err(error),
        }
    }
    
    fn and_then<U, F>(self, f: F) -> Result<U, E>
    where
        F: FnOnce(T) -> Result<U, E>,
    {
        match self {
            Result::Ok(value) => f(value),
            Result::Err(error) => Result::Err(error),
        }
    }
}

// Generic enum for binary tree
#[derive(Debug)]
enum BinaryTree<T> {
    Empty,
    Node {
        value: T,
        left: Box<BinaryTree<T>>,
        right: Box<BinaryTree<T>>,
    },
}

impl<T> BinaryTree<T> {
    fn new() -> Self {
        BinaryTree::Empty
    }
    
    fn leaf(value: T) -> Self {
        BinaryTree::Node {
            value,
            left: Box::new(BinaryTree::Empty),
            right: Box::new(BinaryTree::Empty),
        }
    }
    
    fn node(value: T, left: BinaryTree<T>, right: BinaryTree<T>) -> Self {
        BinaryTree::Node {
            value,
            left: Box::new(left),
            right: Box::new(right),
        }
    }
    
    fn size(&self) -> usize {
        match self {
            BinaryTree::Empty => 0,
            BinaryTree::Node { left, right, .. } => 1 + left.size() + right.size(),
        }
    }
}

impl<T: std::fmt::Display> BinaryTree<T> {
    fn print_inorder(&self) {
        match self {
            BinaryTree::Empty => {},
            BinaryTree::Node { value, left, right } => {
                left.print_inorder();
                print!("{} ", value);
                right.print_inorder();
            },
        }
    }
}

// Generic enum for different data types
#[derive(Debug)]
enum Either<L, R> {
    Left(L),
    Right(R),
}

impl<L, R> Either<L, R> {
    fn is_left(&self) -> bool {
        matches!(self, Either::Left(_))
    }
    
    fn is_right(&self) -> bool {
        matches!(self, Either::Right(_))
    }
    
    fn map_left<T, F>(self, f: F) -> Either<T, R>
    where
        F: FnOnce(L) -> T,
    {
        match self {
            Either::Left(value) => Either::Left(f(value)),
            Either::Right(value) => Either::Right(value),
        }
    }
    
    fn map_right<T, F>(self, f: F) -> Either<L, T>
    where
        F: FnOnce(R) -> T,
    {
        match self {
            Either::Left(value) => Either::Left(value),
            Either::Right(value) => Either::Right(f(value)),
        }
    }
}

fn generic_enums_demo() {
    println!("\n=== Generic Enums ===");
    
    // Custom Result type
    fn divide(a: f64, b: f64) -> Result<f64, String> {
        if b == 0.0 {
            Result::Err("Division by zero".to_string())
        } else {
            Result::Ok(a / b)
        }
    }
    
    let result1 = divide(10.0, 2.0);
    let result2 = divide(10.0, 0.0);
    
    println!("Division results:");
    println!("  10.0 / 2.0 = {:?}", result1);
    println!("  10.0 / 0.0 = {:?}", result2);
    
    // Chain operations with map and and_then
    let chained = divide(20.0, 4.0)
        .map(|x| x * 2.0)
        .and_then(|x| divide(x, 2.0));
    
    println!("  Chained result: {:?}", chained);
    
    // Binary tree
    let tree = BinaryTree::node(
        4,
        BinaryTree::node(
            2,
            BinaryTree::leaf(1),
            BinaryTree::leaf(3)
        ),
        BinaryTree::node(
            6,
            BinaryTree::leaf(5),
            BinaryTree::leaf(7)
        )
    );
    
    println!("\nBinary tree:");
    println!("  Size: {}", tree.size());
    print!("  Inorder traversal: ");
    tree.print_inorder();
    println!();
    
    // Either type for handling different data
    let values: Vec<Either<i32, String>> = vec![
        Either::Left(42),
        Either::Right("Hello".to_string()),
        Either::Left(100),
        Either::Right("World".to_string()),
    ];
    
    println!("\nEither values:");
    for (i, value) in values.iter().enumerate() {
        match value {
            Either::Left(num) => println!("  {}: Number {}", i, num),
            Either::Right(text) => println!("  {}: Text '{}'", i, text),
        }
    }
    
    // Transform Either values
    let transformed: Vec<_> = values.into_iter()
        .map(|either| either.map_left(|n| n * 2).map_right(|s| s.to_uppercase()))
        .collect();
    
    println!("\nTransformed values: {:?}", transformed);
}

generic_enums_demo();

---

## 🎯 Guided Practice

### Exercise 1: Generic Data Structures

Create a generic stack data structure with various operations.

In [ ]:
// TODO: Complete the generic stack implementation

#[derive(Debug)]
struct Stack<T> {
    items: Vec<T>,
}

impl<T> Stack<T> {
    fn new() -> Self {
        // TODO: Create a new empty stack
        Stack { items: Vec::new() }
    }
    
    fn push(&mut self, item: T) {
        // TODO: Push an item onto the stack
        self.items.push(item);
    }
    
    fn pop(&mut self) -> Option<T> {
        // TODO: Pop an item from the stack
        self.items.pop()
    }
    
    fn peek(&self) -> Option<&T> {
        // TODO: Look at the top item without removing it
        self.items.last()
    }
    
    fn is_empty(&self) -> bool {
        // TODO: Check if the stack is empty
        self.items.is_empty()
    }
    
    fn size(&self) -> usize {
        // TODO: Return the number of items in the stack
        self.items.len()
    }
    
    fn clear(&mut self) {
        // TODO: Remove all items from the stack
        self.items.clear();
    }
}

// Additional methods with constraints
impl<T: Clone> Stack<T> {
    fn duplicate_top(&mut self) -> Option<()> {
        // TODO: Duplicate the top item (requires Clone)
        if let Some(top) = self.items.last() {
            let cloned = top.clone();
            self.items.push(cloned);
            Some(())
        } else {
            None
        }
    }
    
    fn to_vec(&self) -> Vec<T> {
        // TODO: Convert stack to vector (top to bottom order)
        let mut result = self.items.clone();
        result.reverse();
        result
    }
}

impl<T: std::fmt::Display> Stack<T> {
    fn display(&self) {
        // TODO: Display the stack contents (requires Display)
        println!("Stack (top to bottom):");
        for (i, item) in self.items.iter().rev().enumerate() {
            println!("  {}: {}", i, item);
        }
        if self.items.is_empty() {
            println!("  (empty)");
        }
    }
}

impl<T: PartialEq> Stack<T> {
    fn contains(&self, item: &T) -> bool {
        // TODO: Check if stack contains an item (requires PartialEq)
        self.items.contains(item)
    }
    
    fn remove_all(&mut self, item: &T) -> usize {
        // TODO: Remove all occurrences of an item, return count removed
        let original_len = self.items.len();
        self.items.retain(|x| x != item);
        original_len - self.items.len()
    }
}

fn stack_demo() {
    println!("\n=== Generic Stack Demo ===");
    
    // Integer stack
    let mut int_stack = Stack::new();
    
    // Push some numbers
    for i in 1..=5 {
        int_stack.push(i);
    }
    
    println!("Integer stack after pushing 1-5:");
    int_stack.display();
    
    // Pop and peek operations
    if let Some(top) = int_stack.peek() {
        println!("\nTop item (peek): {}", top);
    }
    
    if let Some(popped) = int_stack.pop() {
        println!("Popped item: {}", popped);
    }
    
    int_stack.display();
    
    // Duplicate top
    int_stack.duplicate_top();
    println!("\nAfter duplicating top:");
    int_stack.display();
    
    // String stack
    let mut string_stack = Stack::new();
    string_stack.push("first".to_string());
    string_stack.push("second".to_string());
    string_stack.push("third".to_string());
    string_stack.push("second".to_string()); // Duplicate
    
    println!("\nString stack:");
    string_stack.display();
    
    // Check contains and remove
    let search_item = "second".to_string();
    println!("\nContains '{}': {}", search_item, string_stack.contains(&search_item));
    
    let removed_count = string_stack.remove_all(&search_item);
    println!("Removed {} occurrences of '{}'", removed_count, search_item);
    
    string_stack.display();
    
    // Convert to vector
    let as_vec = string_stack.to_vec();
    println!("\nAs vector (top to bottom): {:?}", as_vec);
    
    // Stack statistics
    println!("\nStack statistics:");
    println!("  Size: {}", string_stack.size());
    println!("  Is empty: {}", string_stack.is_empty());
    
    string_stack.clear();
    println!("\nAfter clearing:");
    string_stack.display();
    println!("  Is empty: {}", string_stack.is_empty());
}

stack_demo();

---

## 🧪 Active Recall Checkpoint

**Test your understanding without looking back:**

1. What is monomorphization and how does it affect performance?
2. How do you specify that a generic type must implement certain traits?
3. What's the difference between `<T>` and `<T: Clone>`?
4. When would you use a `where` clause instead of inline bounds?
5. Can you have multiple type parameters in a single generic?
6. How do generic enums like `Option<T>` and `Result<T,E>` work?
7. What happens if you don't use a generic type parameter?
8. How do you implement methods only for specific generic types?

**Write your answers below:**

**Your Answers:**
1. 
2. 
3. 
4. 
5. 
6. 
7. 
8. 

---

## 🤔 Reflection Prompt

Consider these questions:

1. **How do generics help you write more reusable code?**
2. **What are the trade-offs between generics and concrete types?**
3. **How do Rust's generics compare to generics in other languages?**
4. **When might you choose dynamic dispatch over generics?**

Write your thoughts below:

**Your Reflections:**

1. 

2. 

3. 

4. 

---

## 🔮 Preview & Connections

### Coming Up Next: Traits & Shared Behavior

In our next lesson, you'll learn about:
- Defining and implementing traits
- Trait bounds and generic constraints
- Default implementations and associated types
- The orphan rule and trait coherence

### How This Connects
Generics are fundamental to:
- Understanding Rust's type system deeply
- Writing reusable, efficient code
- Working with traits and trait bounds
- Building complex abstractions safely

---

## ✅ Expected Outcomes

**Self-Assessment Checklist** - Can you:

- [ ] Create generic functions that work with multiple types?
- [ ] Define generic structs and enums with type parameters?
- [ ] Apply trait bounds to constrain generic types?
- [ ] Understand monomorphization and its implications?
- [ ] Use where clauses for complex constraints?
- [ ] Implement methods for specific generic instantiations?
- [ ] Build reusable data structures using generics?

If you checked all boxes, excellent! You've mastered Rust's generic system.

---

**🎉 Fantastic Achievement!** You now understand how to write flexible, reusable code with Rust's powerful generic system!